## Perfilamiento del primer algoritmo : Factorizacion PLU

Emplearemos el uso de magic para poder instalar herramientas en el perfilamiento

In [1]:
%pip install -q --user line_profiler

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q --user memory_profiler

  The script mprof is installed in '/Users/pozoshidalgo/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -q --user psutil

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -q --user guppy3

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Para reiniciar el Kernel y cargar los paquete anteriormente instalados
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [3]:
#Librerias a utilizar
import math
import numpy as np
import time
from scipy.linalg import lu
from scipy.integrate import quad

Funciones del algoritmo PLU, hasta el momento tenemos 2: intercambio_filas y nuestro_lup.

In [5]:
def intercambio_filas(M, a, b):
    
    fila_aux = M[a, :].copy()
    M[a, :] =  M[b, :]
    M[b, :] = fila_aux
    return M


def nuestro_lup(A):
    """
     Parameters
    ----------   
    A : ndarray
        2D array containing data with `float` type.
       
    Returns
    -------
    L
        ndarray of matrix L un LUP factorization
    U
        ndarray of matrix U un LUP factorization
    P
        ndarray of matrix P un LUP factorization
    """
    n = len(A)
    L = np.zeros((n,n), dtype=np.float) 
    U = A.copy()
    P =  np.identity(n, dtype=np.float) 

    for col_actual in range(n):
        #imprime_status(col_actual, P,L,U)

        #Get maximum row to swap values 
        row_mayor = col_actual
        a_hat = abs(U[col_actual,col_actual])
        for row in range(col_actual, n): 
            a = abs(U[row, col_actual])
            if a > a_hat : 
                row_mayor = row
                a_hat = a

        #Swap values
        U = intercambio_filas(U, row_mayor, col_actual)
        L = intercambio_filas(L, row_mayor, col_actual)
        P = intercambio_filas(P, row_mayor, col_actual) 


        for j in range(col_actual+1, n):
            vj = U[col_actual, col_actual]
            if (vj !=0):
                multi =  U[j, col_actual] / vj
                L[j, col_actual] =  multi
                U[j, :] = U[j, :] - multi * U[col_actual, :]

    L = L + np.identity(n, dtype=np.float)
    return L, U, P 


Probamos el algoritmo PLU con la siguiente matriz:

$$A = \begin{bmatrix} 1& 4&-2 \\ -3& 9& 8 \\ 5& 1& -6\end{bmatrix}$$

In [10]:
#Ejemplo de matriz de 3x3 a aproximar
A = np.array([[ 1, 4,-2], \
              [-3, 9, 8],
              [ 5, 1,-6]], dtype=np.float)


L, U, P = nuestro_lup(A)

print("Matriz L")
print(L)
print("Matriz U")
print(U)
print("Matriz P")
print(P)

Matriz L
[[ 1.          0.          0.        ]
 [-0.6         1.          0.        ]
 [ 0.2         0.39583333  1.        ]]
Matriz U
[[ 5.          1.         -6.        ]
 [ 0.          9.6         4.4       ]
 [ 0.          0.         -2.54166667]]
Matriz P
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


Para poder evaluar el resultado usaremos el error relativo con la siguiente función.

In [16]:
def err_relativo(aprox, obj):
    return np.fabs(aprox-obj)/np.fabs(obj) #obsérvese el uso de la librería np porque son matrices

In [14]:
#El resultado con el paquete de scipy
p, l, u = lu(A)

print("Matriz l")
print(l)
print("Matriz u")
print(u)
print("Matriz p")
print(p)

Matriz l
[[ 1.          0.          0.        ]
 [-0.6         1.          0.        ]
 [ 0.2         0.39583333  1.        ]]
Matriz u
[[ 5.          1.         -6.        ]
 [ 0.          9.6         4.4       ]
 [ 0.          0.         -2.54166667]]
Matriz p
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [17]:
#Falta definir un error relativo para matrices
err_relativo(L,l)

/Users/pozoshidalgo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


array([[0.00000000e+00,            nan,            nan],
       [1.85037171e-16, 0.00000000e+00,            nan],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

## 1.1 Medición de tiempos

### 1.1.1 Uso de time

In [19]:
start_time = time.time()
aprox=nuestro_lup(A)
end_time = time.time()

In [20]:
secs = end_time-start_time
print("Nuestro algoritmo lup tomó",secs,"segundos" )

Nuestro algoritmo lup tomó 0.0006189346313476562 segundos


Nuestro primer algoritmo tarda muy poco tiempo en su ejecuación con la primer métrica que es $time$. 

### 1.1.2 Uso de /usr/bin/time de Unix

In [28]:
%%file nuestro_lup.py
def nuestro_lup(A):
    """
     Parameters
    ----------   
    A : ndarray
        2D array containing data with `float` type.
       
    Returns
    -------
    L
        ndarray of matrix L un LUP factorization
    U
        ndarray of matrix U un LUP factorization
    P
        ndarray of matrix P un LUP factorization
    """
    n = len(A)
    L = np.zeros((n,n), dtype=np.float) 
    U = A.copy()
    P =  np.identity(n, dtype=np.float) 

    for col_actual in range(n):
        #imprime_status(col_actual, P,L,U)

        #Get maximum row to swap values 
        row_mayor = col_actual
        a_hat = abs(U[col_actual,col_actual])
        for row in range(col_actual, n): 
            a = abs(U[row, col_actual])
            if a > a_hat : 
                row_mayor = row
                a_hat = a

        #Swap values
        U = intercambio_filas(U, row_mayor, col_actual)
        L = intercambio_filas(L, row_mayor, col_actual)
        P = intercambio_filas(P, row_mayor, col_actual) 


        for j in range(col_actual+1, n):
            vj = U[col_actual, col_actual]
            if (vj !=0):
                multi =  U[j, col_actual] / vj
                L[j, col_actual] =  multi
                U[j, :] = U[j, :] - multi * U[col_actual, :]

    L = L + np.identity(n, dtype=np.float)
    return L, U, P 

if __name__=="__main__": #añadimos este bloque para ejecución de la función 
        A = np.array([[ 1, 4,-2], \
              [-3, 9, 8],
              [ 5, 1,-6]], dtype=np.float)
        print("aproximación: {:0.6e}".format(L, U, P = nuestro_lup(A))

Overwriting nuestro_lup.py


In [92]:
##Falta esto
#%%bash
#sudo apt-get install time

### 1.1.3 Uso de %timeit

In [84]:
%timeit -n 8 -r 10 L, U, P = nuestro_lup(A)

172 µs ± 73.5 µs per loop (mean ± std. dev. of 10 runs, 8 loops each)


La función de "nuestro_lup" tiene un loop de tamaño $8$, al repetir esto 10 veces se promedio y se calcula la desviación estandar que tiene mejores resultados. En este caso don 172 microsegundos.

## 1.2 Medición de uso de CPU

### 1.2.1 Uso de cProfile

In [90]:
#Falta esto
#%%bash
#python3 -m cProfile -s cumulative nuestro_lup.py

### 1.2.2 Uso de line_profiler

In [91]:
%load_ext line_profiler

In [97]:
%lprun -f nuestro_lup nuestro_lup(A)

Del algoritmo lo que más ocupa tiempo es el intercambio de filas en las matrices U, L y P probablemente porque mandar a llamar a una función que esta afuera de nuetro_lup. Del mismo modo la línea 54 que es la que resuelve la matriz U ocupa tiempo considerable.

### 1.2.3 line_profiler desde la línea de comandos:

In [99]:
%%file nuestro_lup.py
import numpy as np

def nuestro_lup(A):
    """
     Parameters
    ----------   
    A : ndarray
        2D array containing data with `float` type.
       
    Returns
    -------
    L
        ndarray of matrix L un LUP factorization
    U
        ndarray of matrix U un LUP factorization
    P
        ndarray of matrix P un LUP factorization
    """
    n = len(A)
    L = np.zeros((n,n), dtype=np.float) 
    U = A.copy()
    P =  np.identity(n, dtype=np.float) 

    for col_actual in range(n):
        #imprime_status(col_actual, P,L,U)

        #Get maximum row to swap values 
        row_mayor = col_actual
        a_hat = abs(U[col_actual,col_actual])
        for row in range(col_actual, n): 
            a = abs(U[row, col_actual])
            if a > a_hat : 
                row_mayor = row
                a_hat = a

        #Swap values
        U = intercambio_filas(U, row_mayor, col_actual)
        L = intercambio_filas(L, row_mayor, col_actual)
        P = intercambio_filas(P, row_mayor, col_actual) 


        for j in range(col_actual+1, n):
            vj = U[col_actual, col_actual]
            if (vj !=0):
                multi =  U[j, col_actual] / vj
                L[j, col_actual] =  multi
                U[j, :] = U[j, :] - multi * U[col_actual, :]

    L = L + np.identity(n, dtype=np.float)
    return L, U, P 

if __name__=="__main__": #añadimos este bloque para ejecución de la función 
        A = np.array([[ 1, 4,-2], \
              [-3, 9, 8],
              [ 5, 1,-6]], dtype=np.float)
        print("aproximación: {:0.6e}".format(L, U, P = nuestro_lup(A))

Overwriting nuestro_lup.py


## 1.3 Uso de memoria RAM

### 1.3.1 Uso de %memit

In [103]:
%load_ext memory_profiler

In [104]:
%memit -c L, U, P = nuestro_lup(A)

peak memory: 75.52 MiB, increment: 5.95 MiB


El uso de la memoria es baja para el primer algoritmo desarrollado

### 1.3.2 Uso de memory_profiler

In [ ]:
### Falta

### 1.3.2 Uso de heapy

In [109]:
import math
from guppy import hpy
def nuestro_lup(A):
    """
     Parameters
    ----------   
    A : ndarray
        2D array containing data with `float` type.
       
    Returns
    -------
    L
        ndarray of matrix L un LUP factorization
    U
        ndarray of matrix U un LUP factorization
    P
        ndarray of matrix P un LUP factorization
    """
    n = len(A)
    L = np.zeros((n,n), dtype=np.float) 
    U = A.copy()
    P =  np.identity(n, dtype=np.float)
    
    hp=hpy()
    h=hp.heap()
    print("beginning of PLU factorization")
    print(h)

    for col_actual in range(n):
        #imprime_status(col_actual, P,L,U)

        #Get maximum row to swap values 
        row_mayor = col_actual
        a_hat = abs(U[col_actual,col_actual])
        for row in range(col_actual, n): 
            a = abs(U[row, col_actual])
            if a > a_hat : 
                row_mayor = row
                a_hat = a

        #Swap values
        U = intercambio_filas(U, row_mayor, col_actual)
        L = intercambio_filas(L, row_mayor, col_actual)
        P = intercambio_filas(P, row_mayor, col_actual)
        
        h=hp.heap()
        print("After swiching rows in matrix")
        print(h)


        for j in range(col_actual+1, n):
            vj = U[col_actual, col_actual]
            if (vj !=0):
                multi =  U[j, col_actual] / vj
                L[j, col_actual] =  multi
                U[j, :] = U[j, :] - multi * U[col_actual, :]
                
        h=hp.heap()
        print("After loop of finding U and L")
        print(h)

    L = L + np.identity(n, dtype=np.float)
    return L, U, P 

In [113]:
L, U, P = nuestro_lup(A)

beginning of PLU factorization
Partition of a set of 446293 objects. Total size = 51388840 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 229819  51 22750375  44  22750375  44 str
     1  85342  19  6571760  13  29322135  57 tuple
     2  39758   9  3169213   6  32491348  63 bytes
     3  20050   4  2902248   6  35393596  69 types.CodeType
     4  17849   4  2427464   5  37821060  74 function
     5   2398   1  2313688   5  40134748  78 type
     6   4224   1  1844728   4  41979476  82 dict (no owner)
     7   1103   0  1671320   3  43650796  85 dict of module
     8   2192   0  1538952   3  45189748  88 list
     9   2398   1  1196168   2  46385916  90 dict of type
<875 more rows. Type e.g. '_.more' to view.>
After swiching rows in matrix
Partition of a set of 446335 objects. Total size = 51392216 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 229819  51 22750375  44  22750375  44 str
     1  85347  19  